### How this works - Step by step explanation
1. Diagrams obtained using simulated annealing, coefficients missing (especially also after changing the diagrams to nicer forms, e.g. Hadamard edges)
2. Print matrices of those diagrams
3. Use WolframAlpha to figure out what exact number the obtained numerical values are (usually multiples of sqrt(2))
4. Multiply by the inverse of those numbers exactly (e.g. using g.scalar.add_float(0.125))
5. This gives arrays usually only consisting of ones
6. Now evaluate generated code in Mathematica -> Usually yields integer coefficient solutions
7. Multiply diagrams with those new values -> Finaly coefficients/decomposition
(8. Check decomposition)

In [13]:
import pyzx as zx
from pyzx.basicrules import fuse
from pyzx.tikz import tikz_to_graph
from fractions import Fraction
import numpy as np
import sympy as sp
from collections import defaultdict
import matplotlib.pyplot as plt
import sys

np.set_printoptions(threshold=np.inf) # print the whole matrix

In [14]:
Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
W_INP = zx.VertexType.W_INPUT
W_OUT = zx.VertexType.W_OUTPUT
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [15]:
draw_scale_default = 30

def nice_print(str, conditional=True):
    if conditional:
        print(str)

# like zx.draw but with my favorite settings already set
def nice_draw(g, conditional=True, labels=True, scale=draw_scale_default):
    if not conditional:
        return
    
    exact_string = str(g.scalar).split(" = ")[1]
    numeric_string = str(g.scalar.to_number())
    print("\t" + exact_string + " ≈ " + numeric_string)

    zx.draw(g, labels=labels, scale=scale)

def to_pure_matrix(matrix):
    # get rid of floating point error
    # change -0.0 to 0.0
    return np.around(matrix, 10) + 0.

In [16]:
def tikz_file_to_graph(path):
    with open(path, 'r') as file:
        content = file.read()
    return tikz_to_graph(content)

In [17]:
def vec_to_mathematica_str(name, vec):
    data = repr(vec).strip(u'\u200b')[6:][:-1]

    return "{}=ImportString[\"{}\",\"PythonExpression\"];".format(name, data)

In [18]:
def create_n_states(n, phase):
    goal_graph = zx.Graph()

    for i in range(n):
        cur = zx.Graph()
        
        z = cur.add_vertex(Z, qubit=1, row=0, phase=phase)
        x = cur.add_vertex(X, qubit=1, row=1, phase=Fraction(1,1))
        out = cur.add_vertex(B, qubit=1, row=2)
        cur.add_edge((z,x))
        cur.add_edge((x,out))
        cur.auto_detect_io()

        triangle = zx.generate.spider("W",1,2) + (zx.generate.spider("Z", 1, 0) @ zx.generate.identity(1))
        triangle.auto_detect_io()

        cur = cur + triangle

        goal_graph = goal_graph @ cur
    
    return goal_graph

In [19]:
orig_g = create_n_states(n=4, phase=0.5)

nice_draw(orig_g, labels=False)

orig_matrix = to_pure_matrix(orig_g.to_matrix())

print(orig_matrix)

	sqrt(2)^0 ≈ (1+0j)


[[-4.+0.j]
 [-2.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 1.+1.j]
 [ 1.+0.j]]


In [20]:
g_matrices = []

In [21]:
print(0)
g0 = tikz_file_to_graph("0.tikz")
g0.scalar.add_float(8)
nice_draw(g0, labels=False)
g0.auto_detect_io()
g_matrices.append(to_pure_matrix(g0.to_matrix()))
print(g0.to_matrix())

print(1)
g1 = tikz_file_to_graph("1.tikz")
nice_draw(g1, labels=False)
g1.auto_detect_io()
g_matrices.append(to_pure_matrix(g1.to_matrix()))
print(g1.to_matrix())

print(2)
g2 = tikz_file_to_graph("2.tikz")
g2.scalar.add_float(4)
g2.scalar.add_power(1)
nice_draw(g2, labels=False)
g2.auto_detect_io()
g_matrices.append(to_pure_matrix(g2.to_matrix()))
print(g2.to_matrix())

print(3)
g3 = tikz_file_to_graph("3.tikz")
g3.scalar.add_float(8)
nice_draw(g3, labels=False)
g3.auto_detect_io()
g_matrices.append(to_pure_matrix(g3.to_matrix()))
print(g3.to_matrix())

print(4)
g4 = tikz_file_to_graph("4.tikz")
g4.scalar.add_float(2)
nice_draw(g4, labels=False)
g4.auto_detect_io()
g_matrices.append(to_pure_matrix(g4.to_matrix()))
print(g4.to_matrix())

0
	8.00+0.00isqrt(2)^0 ≈ (8+0j)


[[ 1.+0.j]
 [ 1.+0.j]
 [ 1.+0.j]
 [-1.+0.j]
 [ 1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [ 1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [-1.+0.j]
 [ 1.+0.j]]
1
	sqrt(2)^0 ≈ (1+0j)


[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+1.j]]
2
	4.00+0.00isqrt(2)^1 ≈ (5.656854249492381+0j)


[[-0.+0.j]
 [ 1.-0.j]
 [ 1.-0.j]
 [-0.+0.j]
 [ 1.-0.j]
 [-0.+0.j]
 [ 0.-0.j]
 [-1.+0.j]
 [ 1.-0.j]
 [-0.+0.j]
 [ 0.-0.j]
 [-1.+0.j]
 [ 0.-0.j]
 [-1.+0.j]
 [-1.+0.j]
 [ 0.-0.j]]
3
	8.00+0.00isqrt(2)^0 ≈ (8+0j)


[[1.+0.j]
 [0.+1.j]
 [0.+1.j]
 [1.-0.j]
 [0.+1.j]
 [1.-0.j]
 [1.-0.j]
 [0.+1.j]
 [0.+1.j]
 [1.-0.j]
 [1.-0.j]
 [0.+1.j]
 [1.-0.j]
 [0.+1.j]
 [0.+1.j]
 [1.-0.j]]
4
	2.00+0.00isqrt(2)^0 ≈ (2+0j)


[[0.+0.j]
 [1.-0.j]
 [1.-0.j]
 [0.+0.j]
 [1.-0.j]
 [0.+0.j]
 [0.+0.j]
 [1.-0.j]
 [1.-0.j]
 [0.+0.j]
 [0.+0.j]
 [1.-0.j]
 [0.+0.j]
 [1.-0.j]
 [1.-0.j]
 [0.+0.j]]


In [22]:
mats = g_matrices

goal_str = vec_to_mathematica_str("Goal", orig_matrix.flatten())
mat_strs = []
solve_str = "Solve[m0*M0"

for i, mat in enumerate(mats):
    mat_strs.append(vec_to_mathematica_str("M" + str(i), mat))

    if i > 0:
        solve_str += " + m{}*M{}".format(i, i)

solve_str += " == Goal, {m0"

for i in range(1, len(mats)):
    solve_str += ", m{}".format(i)

solve_str += "}]"

print(goal_str)
for mat_str in mat_strs:
    print(mat_str)
print(solve_str)

Goal=ImportString["[-4.+0.j, -2.+2.j, -2.+2.j,  0.+2.j, -2.+2.j,  0.+2.j,  0.+2.j,
        1.+1.j, -2.+2.j,  0.+2.j,  0.+2.j,  1.+1.j,  0.+2.j,  1.+1.j,
        1.+1.j,  1.+0.j]","PythonExpression"];
M0=ImportString["[[ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [ 1.+0.j]]","PythonExpression"];
M1=ImportString["[[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+1.j]]","PythonExpression"];
M2=ImportString["[[ 0.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 0.+0.j],
       [ 1.+0.j],
       [ 0.+0.j],
       [ 0.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
 

In [23]:
g_matrices = []

print(0)
g0 = tikz_file_to_graph("0.tikz")
g0.scalar.add_float(8)
g0.scalar.add_float(-0.75+0.25j)
nice_draw(g0, labels=False)
g0.auto_detect_io()
g_matrices.append(to_pure_matrix(g0.to_matrix()))
print(g0.to_matrix())

print(1)
g1 = tikz_file_to_graph("1.tikz")
g1.scalar.add_float(-2.5-2.5j)
nice_draw(g1, labels=False)
g1.auto_detect_io()
g_matrices.append(to_pure_matrix(g1.to_matrix()))
print(g1.to_matrix())

print(2)
g2 = tikz_file_to_graph("2.tikz")
g2.scalar.add_float(4)
g2.scalar.add_power(1)
g2.scalar.add_float(-0.75+0.25j)
nice_draw(g2, labels=False)
g2.auto_detect_io()
g_matrices.append(to_pure_matrix(g2.to_matrix()))
print(g2.to_matrix())

print(3)
g3 = tikz_file_to_graph("3.tikz")
g3.scalar.add_float(8)
g3.scalar.add_float(-0.75+2.25j)
nice_draw(g3, labels=False)
g3.auto_detect_io()
g_matrices.append(to_pure_matrix(g3.to_matrix()))
print(g3.to_matrix())

print(4)
g4 = tikz_file_to_graph("4.tikz")
g4.scalar.add_float(2)
g4.scalar.add_float(1.75+2.25j)
nice_draw(g4, labels=False)
g4.auto_detect_io()
g_matrices.append(to_pure_matrix(g4.to_matrix()))
print(g4.to_matrix())

0
	-6.00+2.00isqrt(2)^0 ≈ (-6+2j)


[[-0.75+0.25j]
 [-0.75+0.25j]
 [-0.75+0.25j]
 [ 0.75-0.25j]
 [-0.75+0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [-0.75+0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [-0.75+0.25j]]
1
	-2.50-2.50isqrt(2)^0 ≈ (-2.5-2.5j)


[[-2.5-2.5j]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 0. -0.j ]
 [ 2.5-2.5j]]
2
	-3.00+1.00isqrt(2)^1 ≈ (-4.242640687119286+1.4142135623730951j)


[[-0.  -0.j  ]
 [-0.75+0.25j]
 [-0.75+0.25j]
 [-0.  -0.j  ]
 [-0.75+0.25j]
 [-0.  -0.j  ]
 [ 0.  +0.j  ]
 [ 0.75-0.25j]
 [-0.75+0.25j]
 [-0.  -0.j  ]
 [ 0.  +0.j  ]
 [ 0.75-0.25j]
 [ 0.  +0.j  ]
 [ 0.75-0.25j]
 [ 0.75-0.25j]
 [ 0.  +0.j  ]]
3
	-6.00+18.00isqrt(2)^0 ≈ (-6+18j)


[[-0.75+2.25j]
 [-2.25-0.75j]
 [-2.25-0.75j]
 [-0.75+2.25j]
 [-2.25-0.75j]
 [-0.75+2.25j]
 [-0.75+2.25j]
 [-2.25-0.75j]
 [-2.25-0.75j]
 [-0.75+2.25j]
 [-0.75+2.25j]
 [-2.25-0.75j]
 [-0.75+2.25j]
 [-2.25-0.75j]
 [-2.25-0.75j]
 [-0.75+2.25j]]
4
	3.50+4.50isqrt(2)^0 ≈ (3.5+4.5j)


[[-0.  +0.j  ]
 [ 1.75+2.25j]
 [ 1.75+2.25j]
 [-0.  +0.j  ]
 [ 1.75+2.25j]
 [-0.  +0.j  ]
 [-0.  +0.j  ]
 [ 1.75+2.25j]
 [ 1.75+2.25j]
 [-0.  +0.j  ]
 [-0.  +0.j  ]
 [ 1.75+2.25j]
 [-0.  +0.j  ]
 [ 1.75+2.25j]
 [ 1.75+2.25j]
 [-0.  +0.j  ]]


In [24]:
final_matrix = np.zeros((2**4, 1), dtype=complex)

for m in g_matrices:
    final_matrix += m

print(orig_matrix)
print(final_matrix)

print(np.all(orig_matrix==final_matrix))

[[-4.+0.j]
 [-2.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 1.+1.j]
 [ 1.+0.j]]
[[-4.+0.j]
 [-2.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [-2.+2.j]
 [ 0.+2.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 0.+2.j]
 [ 1.+1.j]
 [ 1.+1.j]
 [ 1.+0.j]]
True


# Phase